In [28]:
import pandas as pd
df = pd.read_csv('iphone.csv')

#### 1) The column names have spaces . rename the column names to have underscore '_' instead of space (try to do in one go instead of specifying each column nam in rename method)

In [29]:
df.columns = df.columns.str.replace(' ', '_')

In [39]:
df = pd.read_csv('iphone.csv')
# #df.columns[9]='RAM' # columns are immutable hence we use rename method
# df.rename(columns={df.columns[9]: 'RAM'}, inplace=True)
# #method 2: Assigning a New List of Column Names
# columns = list(df.columns)
# columns[9] = 'RAM'
# df.columns = columns

#### 2) start rating for some of the models is missing in the dataset. fill those missing values with the average rating all the models.

In [30]:
average_rating = df['Star_Rating'].mean(skipna=True) #excluding None (missing values)
df['Star_Rating'].fillna(average_rating)#, inplace=True)

0     4.600000
1     4.600000
2     4.600000
3     4.576271
4     4.500000
        ...   
57    4.500000
58    4.600000
59    4.600000
60    4.600000
61    4.600000
Name: Star_Rating, Length: 62, dtype: float64

#### 3) Now instead of filling missing values with avg rating of full dataset , fill with avg rating based on RAM. example :  if rating for a 2 gb phone is missing then take average of all other 2 gb phones rating and fill that value. 

In [130]:
df.groupby('RAM').agg({'Star Rating':'mean'}).to_dict()

{'Star Rating': {'2 GB': 4.541666666666666,
  '3 GB': 4.6,
  '4 GB': 4.589285714285714,
  '6 GB': 4.5777777777777775}}

In [129]:
#Method 1
# Calculate the average rating for each RAM group
average_ratings_by_ram = df.groupby('RAM')['Star Rating'].mean().reset_index()

# Rename the 'Star Rating' column to 'Average Rating' for clarity
average_ratings_by_ram.rename(columns={'Star Rating': 'Average Rating'}, inplace=True)

# Perform a left join to fill missing ratings based on RAM
df = df.merge(average_ratings_by_ram, on='RAM', how='left')

# Fill missing ratings with the calculated average ratings
df['Star Rating'].fillna(df['Average Rating'], inplace=True)

#Drop the 'Average Rating' column if not needed
df.drop(columns=['Average Rating'], inplace=True)

In [35]:
df = pd.read_csv('iphone.csv')

In [135]:
#Method 2
# Calculate the average rating for each RAM group
average_ratings_by_ram = df.groupby('RAM')['Star Rating'].mean().to_dict()

# Function to fill missing ratings based on RAM
def fill_missing_rating(row):
    if pd.isna(row['Star Rating']):
        return average_ratings_by_ram.get(row['RAM'], None)
    return row['Star Rating']

# Apply the function to fill missing ratings
df['Star Rating'] = df.apply(fill_missing_rating, axis=1)

#### 4) create a new column in the dataframe "Discount_Percentage" based on MRP and sale value

In [136]:
df['Discount Percentage'] = ((df['Mrp'] - df['Sale Price']) / df['Mrp']) * 100

#### 5) which model has highest percent discount ?

In [138]:
#method1
df[df['Discount Percentage']==df['Discount Percentage'].max()]

,Product Name,Product URL,Brand,Sale Price,Mrp,Number Of Ratings,Number Of Reviews,Upc,Star Rating,RAM,Discount Percentage,space configuration,models,iPhone Version,iPhone Category,Color
18,"APPLE iPhone 11 Pro (Midnight Green, 64 GB)",https://www.flipkart.com/apple-iphone-11-pro-m...,Apple,74999,106600,7088,523,MOBFKCTSN3TG3RFJ,4.6,4 GB,29.644465,64 GB,11,iPhone 11,iPhone 11,Midnight Green


In [47]:
#method2
sorted_df = df.sort_values(by='Discount Percentage', ascending=False)
model_with_highest_discount = sorted_df.iloc[0]['Product Name']
highest_discount_percentage = sorted_df.iloc[0]['Discount Percentage']
model_with_highest_discount,highest_discount_percentage

('APPLE iPhone 11 Pro (Midnight Green, 64 GB)', 29.644465290806753)

#### 6) find total no of models  each space configuration (128 GB , 64 GB etc)

In [143]:
df['Product Name'][0].split(',')[1].split(')')[0].strip()

'64 GB'

In [144]:
df['space configuration'] = df['Product Name'].str.split(',').str[1].str.split(')').str[0].str.strip()
df['space configuration'].value_counts()

128 GB    24
64 GB     20
256 GB    14
512 GB     4
Name: space configuration, dtype: int64

In [146]:
# using str.extract method with a regular expression.
df['Product Name'].str.extract(r'\(([^,]+), (\d+ GB)\)', expand=True)[1]

0      64 GB
1     256 GB
2     256 GB
3     256 GB
4     256 GB
       ...  
57     64 GB
58     64 GB
59     64 GB
60     64 GB
61     64 GB
Name: 1, Length: 62, dtype: object

In [52]:
space_configuration_counts = df['RAM'].value_counts()
# Rename the columns for clarity
space_configuration_counts.columns = ['Space Configuration', 'Total Models']
space_configuration_counts

4 GB    29
6 GB    19
2 GB    13
3 GB     1
Name: RAM, dtype: int64

# Regular Expressions

***Literal Characters:*** These are the simplest form of regex. For example, the regex cat will match the string “cat” in any text.

***Metacharacters:*** These are special characters that have a specific meaning in regex. Some common ones are:\
***.***: Matches any single character except newline.\
***^***: Matches the start of a string.\
***$***: Matches the end of a string.\
*: Matches 0 or more repetitions of the preceding element.\
***+***: Matches 1 or more repetitions of the preceding element.\
***?***: Matches 0 or 1 repetition of the preceding element.

***Character Classes:*** These allow you to match any one of a set of characters.\
***[abc]***: Matches any one of the characters a, b, or c.\
***[a-z]***: Matches any one character from a to z.\
***[^abc]***: Matches any character except a, b, or c.

***Quantifiers:*** These specify how many times the preceding element should be matched.\
***{n}***: Matches exactly n times.\
***{n,}***: Matches at least n times.\
***{n,m}***: Matches between n and m times.

***Groups and Alternation:*** \
***(...)***: Groups multiple tokens together and remembers the matched text.\
***|***: Acts like a logical OR to match one pattern or another.

**Examples**

***Simple Match:*** \
Regex: cat\
Matches: “cat” in “The cat sat on the mat.”

***Any Character:*** \
Regex: c.t\
Matches: “cat”, “cot”, “cut” in “The cat sat on the cot and cut the mat.”

***Start and End:*** \
Regex: ^cat\
Matches: “cat” only if it is at the start of the string.

Regex: cat$\
Matches: “cat” only if it is at the end of the string.

***Character Classes:*** \
Regex: [aeiou]\
Matches: Any vowel in the text.

***Quantifiers:*** \
Regex: a{2,4}\
Matches: “aa”, “aaa”, or “aaaa” in the text.

In [ ]:
import re

text = "APPLE iPhone 8 Plus (Gold, 64 GB)"
match = re.search(r'\(([^,]+), (\d+ GB)\)', text)
if match:
    space_config = match.group(2)
    print(space_config)  # Output: 64 GB
    
# r before the string denotes a raw string => (\) are treated as literal characters and not as escape characters.   
# \( and \): Match the literal parentheses.
# ([^,]+): Matches any characters except a comma, capturing the color.
# (\d+ GB): Matches one or more digits followed by " GB", capturing the space configuration.

In [ ]:
text = "APPLE iPhone 8 Plus (Gold, 64 GB)"
match = re.search(r'\(([^,]+), (\d+ GB)\)', text)
if match:
    space_config = match.group(1)
    print(space_config)  # Output: 64 GB

In [ ]:
import re

text = "APPLE iPhone 8 Plus (Gold, 64 GB)"
pattern = r'\(([^)]+)\)'

match = re.search(pattern, text)
if match:
    content = match.group(1)
    print(content)  # This should print: Gold, 64 GB
else:
    print("No match found")

#### 7) find total number of models for each color 

In [110]:
df['Product Name'].str.split('(').str[1].str.split(',').str[0].value_counts()

Black             10
White             10
Silver             7
Gold               6
Space Grey         6
Red                5
Midnight Green     4
Pacific Blue       4
Graphite           4
Blue               2
RED                1
Coral              1
Green              1
Purple             1
Name: Product Name, dtype: int64

In [114]:
df['Product Name'].str.extract(r'\(([^,]+), (\d+ GB)\)', expand=True)

,0,1
0,Gold,64 GB
1,Space Grey,256 GB
2,Silver,256 GB
3,Silver,256 GB
4,Gold,256 GB
...,...,...
57,Black,64 GB
58,Purple,64 GB
59,White,64 GB
60,Black,64 GB


In [115]:
# Define a function to extract color based on a pattern
def extract_color(product_name):
    # Split the product name by '(' and ')' to get parts within parentheses
    parts = product_name.split('(')
    if len(parts) > 1:
        # Check if the second part contains a comma (,) to identify color
        color_part = parts[1].strip()
        if ',' in color_part:
            color = color_part.split(',')[0].strip()
            return color
    return None

# Apply the extract_color function to create a 'Color' column
df['Color'] = df['Product Name'].apply(extract_color)

# Count the number of models for each color
color_counts = df['Color'].value_counts().reset_index()

# Rename the columns for clarity
color_counts.columns = ['Color', 'Total Models']

# Display the result
print(color_counts)

             Color  Total Models
0            Black            10
1            White            10
2           Silver             7
3             Gold             6
4       Space Grey             6
5              Red             5
6   Midnight Green             4
7     Pacific Blue             4
8         Graphite             4
9             Blue             2
10             RED             1
11           Coral             1
12           Green             1
13          Purple             1


#### 8) find total number of models by iphone version : eg- iphone 8:  9,  iphone XR : 5   so on..

In [81]:
df['Product Name'].str.split().str[2].value_counts()

12    26
11    16
8      8
SE     6
XR     5
XS     1
Name: Product Name, dtype: int64

In [84]:
df['models'] = df['Product Name'].str.split().str[2]

In [85]:
# Define a function to extract iPhone version
def extract_iphone_version(product_name):
    # Split the product name by 'iPhone' and get the part after 'iPhone'
    parts = product_name.split('iPhone')
    if len(parts) > 1:
        version = parts[1].strip().split(' ')[0]
        return 'iPhone ' + version
    return None

# Apply the extract_iphone_version function to create an 'iPhone Version' column
df['iPhone Version'] = df['Product Name'].apply(extract_iphone_version)

# Count the number of models for each iPhone version
iphone_version_counts = df['iPhone Version'].value_counts().reset_index()

# Rename the columns for clarity
iphone_version_counts.columns = ['iPhone Version', 'Total Models']

# Display the result
print(iphone_version_counts)

  iPhone Version  Total Models
0      iPhone 12            26
1      iPhone 11            16
2       iPhone 8             8
3      iPhone SE             6
4      iPhone XR             5
5      iPhone XS             1


#### 9) list top 5 models having highest no of reviews 

In [88]:
df.groupby('models').agg({'Number Of Reviews': sum}).sort_values(by='Number Of Reviews',ascending=False).head(5)

,Number Of Reviews
models,
SE,48952
XR,33988
11,25965
8,4162
12,2208


#### 10) what is the price diffrence between highest price and lowest price iphone (based on mrp)

In [102]:
max_mrp= df.sort_values(by='Mrp').tail(1)['Mrp'].values[0] 
min_mrp= df.sort_values(by='Mrp').head(1)['Mrp'].values[0]
max_mrp-min_mrp
#values[0] part extracts the actual value from the resulting Series.

110000

In [100]:
# Find the highest and lowest MRP values
highest_mrp = df['Mrp'].max()
lowest_mrp = df['Mrp'].min()
price_difference = highest_mrp - lowest_mrp
price_difference

110000

#### 11) find total no of reviews for iphone 11 and iphone 12 category . Output should have only 2 rows (for 11 and 12).

In [147]:
df[df['models'].isin(['11', '12'])].groupby('models').agg({'Number Of Reviews': 'sum'}).reset_index()

,models,Number Of Reviews
0,11,25965
1,12,2208


In [107]:
#method1:
# Categorize products based on 'Product Name' using str.contains
df['iPhone Category'] = 'Other'
df.loc[df['Product Name'].str.contains('iPhone 11', case=False), 'iPhone Category'] = 'iPhone 11'
df.loc[df['Product Name'].str.contains('iPhone 12', case=False), 'iPhone Category'] = 'iPhone 12'

# Group by 'iPhone Category' and calculate the total number of reviews for each category
category_reviews = df.groupby('iPhone Category')['Number Of Reviews'].sum().reset_index()

# Filter for 'iPhone 11' and 'iPhone 12' categories
result_df = category_reviews[category_reviews['iPhone Category'].isin(['iPhone 11', 'iPhone 12'])]

# Display the result
print(result_df)
#In this code, we use str.contains to categorize products based on the 'Product Name' column without using lambda or numpy. The rest of the process remains the same, resulting in a DataFrame with two rows for 'iPhone 11' and 'iPhone 12' categories.


  iPhone Category  Number Of Reviews
1       iPhone 11              25965
2       iPhone 12               2208


In [108]:
#method2:

# Filter the DataFrame for iPhone 11 and iPhone 12 categories
iphone_11_reviews = df[df['Product Name'].str.contains('iPhone 11', case=False)]
iphone_12_reviews = df[df['Product Name'].str.contains('iPhone 12', case=False)]

# Calculate the total number of reviews for each category
total_reviews_11 = iphone_11_reviews['Number Of Reviews'].sum()
total_reviews_12 = iphone_12_reviews['Number Of Reviews'].sum()

# Create a new DataFrame with the results
result_df = pd.DataFrame({
    'iPhone Category': ['iPhone 11', 'iPhone 12'],
    'Total Reviews': [total_reviews_11, total_reviews_12]
})
result_df

,iPhone Category,Total Reviews
0,iPhone 11,25965
1,iPhone 12,2208


#### 12) which iphone has 3rd highest MRP

In [78]:
# Sort the DataFrame by 'Mrp' column in descending order and reset the index
sorted_df = df.sort_values(by='Mrp', ascending=False).reset_index(drop=True)

# Get the iPhone with the 3rd highest MRP
third_highest_mrp_iphone = sorted_df.loc[2]

# Print the details of the iPhone with the 3rd highest MRP
print(third_highest_mrp_iphone)

Product Name                APPLE iPhone 11 Pro (Midnight Green, 512 GB)
Product URL            https://www.flipkart.com/apple-iphone-11-pro-m...
Brand                                                              Apple
Sale Price                                                        117900
Mrp                                                               140300
Number Of Ratings                                                   7088
Number Of Reviews                                                    523
Upc                                                     MOBFKCTSSJCWYGCC
Star Rating                                                          4.6
RAM                                                                 4 GB
Discount Percentage                                            15.965788
space configuration                                               512 GB
Name: 2, dtype: object


#### 13) what is the average mrp of iphones which costs above 100,000

In [76]:
df[df['Mrp']>100000]['Mrp'].mean()

130559.09090909091

#### 14) which iphone with 128 GB space has highest ratings to review ratio

In [70]:
df_128 = df[df['space configuration'] == '128 GB'].copy()
df_128['r2r'] = df_128['Number Of Ratings'] / df_128['Number Of Reviews']
df_128.sort_values(by='r2r',ascending =False).head(1)

,Product Name,Product URL,Brand,Sale Price,Mrp,Number Of Ratings,Number Of Reviews,Upc,Star Rating,RAM,Discount Percentage,space configuration,r2r
56,"APPLE iPhone 11 (Black, 128 GB)",https://www.flipkart.com/apple-iphone-11-black...,Apple,54999,59900,43470,3331,MOBFWQ6BKRYBP5X8,4.6,4 GB,8.18197,128 GB,13.050135


### idxmax(axis=0, skipna=True)

In [75]:
# Filter the DataFrame to include only iPhones with 128 GB of space
iphone_128gb = df[df['space configuration'] == '128 GB'].copy()

# Calculate the ratings-to-reviews ratio for each iPhone
iphone_128gb['Ratings to Reviews Ratio'] = iphone_128gb['Number Of Ratings'] / iphone_128gb['Number Of Reviews']

# Find the iPhone with the highest ratings-to-reviews ratio
highest_ratio_iphone = iphone_128gb.loc[iphone_128gb['Ratings to Reviews Ratio'].idxmax()] #idxmax returns the index of max
highest_ratio_iphone

Product Name                                  APPLE iPhone 11 (Black, 128 GB)
Product URL                 https://www.flipkart.com/apple-iphone-11-black...
Brand                                                                   Apple
Sale Price                                                              54999
Mrp                                                                     59900
Number Of Ratings                                                       43470
Number Of Reviews                                                        3331
Upc                                                          MOBFWQ6BKRYBP5X8
Star Rating                                                               4.6
RAM                                                                      4 GB
Discount Percentage                                                   8.18197
space configuration                                                    128 GB
Ratings to Reviews Ratio                                        